In [1]:
%matplotlib inline
import numpy as np
import os
import time
import h5py
import keras
import pandas as pd
import math
import joblib
import json
import matplotlib.pyplot as plt
from scipy.stats import logistic

from IPython.display import display

from keras.layers import (Input, Dense, Lambda, Flatten, Reshape, BatchNormalization, 
                          Activation, Dropout, Conv2D, Conv2DTranspose,
                          Concatenate, Add, Multiply)
from keras.engine import InputSpec
from keras.losses import sparse_categorical_crossentropy
from keras.optimizers import RMSprop, Adam
from keras.models import Model
from keras import metrics
from keras import backend as K
from keras_tqdm import TQDMNotebookCallback
from keras.datasets import cifar10

import tensorflow as tf

from pixelcnn_helpers import PixelConv2D

/home/brian/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Model

In [2]:
print("Building conv layers...")

def build_test_network(ptype, input_shape=(3, 3, 3)):
    model_in = Input(shape=input_shape, name='main_input')
    model_out = PixelConv2D(ptype, filters=1, kernel_size=3, 
                            padding='same',
                            kernel_initializer='ones', 
                            bias_initializer='zeros')(model_in)
    model = Model(inputs=model_in, outputs=model_out)
    
    optimizer = Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    return model


Building conv layers...


In [3]:
# Test cases
test_ones = np.ones([1, 3, 3, 3])
test_ones_depth6 = np.stack([
    np.ones([1, 3, 3]),
    np.ones([1, 3, 3]) * 0.1,
    np.ones([1, 3, 3]),
    np.ones([1, 3, 3]) * 0.1,
    np.ones([1, 3, 3]),
    np.ones([1, 3, 3]) * 0.1,
], axis=-1)

# Test "r" configuration

In [4]:
print('RA CONFIG depth=3')
model = build_test_network('ra')
val = model.predict(test_ones).reshape(3, 3)

expected = np.array([0, 1, 1, 2, 4, 3, 2, 4, 3]).reshape(3, 3)
display(expected, val)
assert np.array_equal(val, expected)

print('RB CONFIG depth=3')
model = build_test_network('rb')
val = model.predict(test_ones).reshape(3, 3)
expected = expected + 1
display(expected, val)
assert np.array_equal(val, expected)

RA CONFIG depth=3


array([[0, 1, 1],
       [2, 4, 3],
       [2, 4, 3]])

array([[0., 1., 1.],
       [2., 4., 3.],
       [2., 4., 3.]], dtype=float32)

RB CONFIG depth=3


array([[1, 2, 2],
       [3, 5, 4],
       [3, 5, 4]])

array([[1., 2., 2.],
       [3., 5., 4.],
       [3., 5., 4.]], dtype=float32)

In [5]:
print('RA CONFIG depth=6')
model = build_test_network('ra', input_shape=(3, 3, 6))
val = model.predict(test_ones_depth6).reshape(3, 3)

expected = np.array([0, 1.1, 1.1, 2.2, 4.4, 3.3, 2.2, 4.4, 3.3]).reshape(3, 3)
display(expected, val)
assert np.allclose(val, expected)

print('RB CONFIG depth=6')
model = build_test_network('rb', input_shape=(3, 3, 6))
val = model.predict(test_ones_depth6).reshape(3, 3)
expected = expected + 1.1
display(expected, val)
assert np.allclose(val, expected)

RA CONFIG depth=6


array([[0. , 1.1, 1.1],
       [2.2, 4.4, 3.3],
       [2.2, 4.4, 3.3]])

array([[0.       , 1.1      , 1.1      ],
       [2.1999998, 4.3999996, 3.2999997],
       [2.1999998, 4.3999996, 3.2999997]], dtype=float32)

RB CONFIG depth=6


array([[1.1, 2.2, 2.2],
       [3.3, 5.5, 4.4],
       [3.3, 5.5, 4.4]])

array([[1.1      , 2.1999998, 2.1999998],
       [3.2999997, 5.4999995, 4.3999996],
       [3.2999997, 5.4999995, 4.3999996]], dtype=float32)

# Test "g" configuration

In [6]:
print('GA CONFIG')
model = build_test_network('ga')
val = model.predict(test_ones).reshape(3, 3)

expected = np.array([1, 3, 3, 5, 9, 7, 5, 9, 7]).reshape(3, 3)
display(expected, val)
assert np.array_equal(val, expected)

print('GB CONFIG')
model = build_test_network('gb')
val = model.predict(test_ones).reshape(3, 3)

expected = expected + 1
display(expected, val)
assert np.array_equal(val, expected)

GA CONFIG


array([[1, 3, 3],
       [5, 9, 7],
       [5, 9, 7]])

array([[1., 3., 3.],
       [5., 9., 7.],
       [5., 9., 7.]], dtype=float32)

GB CONFIG


array([[ 2,  4,  4],
       [ 6, 10,  8],
       [ 6, 10,  8]])

array([[ 2.,  4.,  4.],
       [ 6., 10.,  8.],
       [ 6., 10.,  8.]], dtype=float32)

In [7]:
print('GA CONFIG depth=6')
model = build_test_network('ga', input_shape=(3, 3, 6))
val = model.predict(test_ones_depth6).reshape(3, 3)

expected = np.array([1.1, 3.3, 3.3, 5.5, 9.9, 7.7, 5.5, 9.9, 7.7]).reshape(3, 3)
display(expected, val)
assert np.allclose(val, expected)

print('GB CONFIG depth=6')
model = build_test_network('gb', input_shape=(3, 3, 6))
val = model.predict(test_ones_depth6).reshape(3, 3)
expected = expected + 1.1
display(expected, val)
assert np.allclose(val, expected)

GA CONFIG depth=6


array([[1.1, 3.3, 3.3],
       [5.5, 9.9, 7.7],
       [5.5, 9.9, 7.7]])

array([[1.1      , 3.2999997, 3.2999997],
       [5.4999995, 9.900002 , 7.6999993],
       [5.4999995, 9.900002 , 7.6999993]], dtype=float32)

GB CONFIG depth=6


array([[ 2.2,  4.4,  4.4],
       [ 6.6, 11. ,  8.8],
       [ 6.6, 11. ,  8.8]])

array([[ 2.1999998,  4.3999996,  4.3999996],
       [ 6.5999994, 11.000002 ,  8.800001 ],
       [ 6.5999994, 11.000002 ,  8.800001 ]], dtype=float32)

# Test "b" configuration

In [8]:
print('BA CONFIG')
model = build_test_network('ba')
val = model.predict(test_ones).reshape(3, 3)

expected = np.array([2, 5, 5, 8, 14, 11, 8, 14, 11]).reshape(3, 3)
display(expected, val)
assert np.array_equal(val, expected)

print('BB CONFIG')
model = build_test_network('bb')
val = model.predict(test_ones).reshape(3, 3)

expected = expected + 1
display(expected, val)
assert np.array_equal(val, expected)

BA CONFIG


array([[ 2,  5,  5],
       [ 8, 14, 11],
       [ 8, 14, 11]])

array([[ 2.,  5.,  5.],
       [ 8., 14., 11.],
       [ 8., 14., 11.]], dtype=float32)

BB CONFIG


array([[ 3,  6,  6],
       [ 9, 15, 12],
       [ 9, 15, 12]])

array([[ 3.,  6.,  6.],
       [ 9., 15., 12.],
       [ 9., 15., 12.]], dtype=float32)

In [9]:
print('BA CONFIG depth=6')
model = build_test_network('ba', input_shape=(3, 3, 6))
val = model.predict(test_ones_depth6).reshape(3, 3)

expected = np.array([2.2, 5.5, 5.5, 8.8, 15.4, 12.1, 8.8, 15.4, 12.1]).reshape(3, 3)
display(expected, val)
assert np.allclose(val, expected)

print('BB CONFIG depth=6')
model = build_test_network('bb', input_shape=(3, 3, 6))
val = model.predict(test_ones_depth6).reshape(3, 3)
expected = expected + 1.1
display(expected, val)
assert np.allclose(val, expected)

BA CONFIG depth=6


array([[ 2.2,  5.5,  5.5],
       [ 8.8, 15.4, 12.1],
       [ 8.8, 15.4, 12.1]])

array([[ 2.1999998,  5.4999995,  5.4999995],
       [ 8.8      , 15.400003 , 12.100002 ],
       [ 8.8      , 15.400003 , 12.100002 ]], dtype=float32)

BB CONFIG depth=6


array([[ 3.3,  6.6,  6.6],
       [ 9.9, 16.5, 13.2],
       [ 9.9, 16.5, 13.2]])

array([[ 3.2999997,  6.5999994,  6.5999994],
       [ 9.900001 , 16.500004 , 13.200003 ],
       [ 9.900001 , 16.500004 , 13.200003 ]], dtype=float32)

# Notes - 2019-04-29

Done:

* Got some simple testing on the Conv2D layer using a 3x3x3 image of "1"s and by setting the Conv2D kernel to be exactly 1.  Verified the "{r,g,b}x{a,b}" operations are mostly working.

TODO:

* Test same thing on a 3x3x6 image (only for {r,g,b}x{b}), which should simulate what happens in the internal layers of a PixelCNN network, where you start having more filters. 
  * Will need to use something more complex than an image of "1"s because you want to test that it picks up the right input depth layers when building the mask.
  * I think it should be stacked "RRGGBB", but check the implementation